In [6]:
import pydicom
from glob import glob
import pandas as pd
from tqdm.contrib.concurrent import process_map
import numpy as np
from typing import Sequence, Any
from tqdm import tqdm
from pydicom.multival import MultiValue
from pydicom.valuerep import DSfloat, DSdecimal, IS

BASE_DIR = "/data/covid_ct"

In [7]:
dcms = list(glob(f"{BASE_DIR}/**/**.dcm"))

In [8]:
def parse_field(data: pydicom.Dataset, field: str) -> Any:
    raw = getattr(data, field)
    if isinstance(raw, MultiValue):
        if raw.type_constructor in [DSfloat, DSdecimal]:
            return tuple(float(i) for i in raw)
        elif raw.type_constructor == IS:
            return tuple(int(i) for i in raw)
        else:
            return tuple(raw)

    return raw

def parse(file: str):
    data = pydicom.dcmread(file)
    return {i: parse_field(data, i) for i in data.dir()}


In [10]:
df = pd.DataFrame.from_records([process_map(parse, dcms, chunksize=50)])

  0%|          | 0/482874 [00:00<?, ?it/s]

In [1]:
for col in df.columns:
    if df[col].nunique() < 10:
        print(f"{col}: {df[col].unique()}")
    else:
        print(f"NOOOOOOOOO {col}")

NameError: name 'df' is not defined